# Hotel Database
## Assignment

Design, populate, and query a database for a hotel reservation system with the following business rules:

1. The hotel has a number of rooms of two types: Deluxe and Suite
2. For every night, some rooms are made available for reservation for a specific price.
3. A guest can make a reservation for an avavilable room for one night. The reservation must include credit card payment info. At most one reservation can be made per night per room.
4. A guest can check into a room that has been reserved. An attempt to check in without a reservation will generate an error.
5. A guest can check out only after checking in. An attempt to check out multiple times or check out without checking in will generate an error.

Your Python code should provide the following:

1. A section to create the tables. The design must be in 3rd normal form following the conventions discussed in class and enforcing the business rules above.

2. Provide code to populate rooms and room availability with prices.

3. The function `reserve_room(room, date, guest_name, credit_card)` to make a reservation. A script that populates at least 300 reservations (e.g. use `faker`)

4. The functions `checkin(room, date)` and `checkout(room, date)` to check guests in and out. Write a script that invokes `checkin` and `checkout` for a buncha guests.  Demonstrate that that the functions enforces the rules of the business.

5. Write a query to list all guests who have stayed in a given room in 2021.

6. Write a query to list all dates on which a specific guest stayed at the hotel.



# Define the database

In [1]:
import datajoint as dj
schema = dj.Schema('shared_hotel')

Connecting llewis1@db.ust-db.link:3306


In [2]:
@schema
class Room(dj.Manual):
    definition = """
    room : int
    --- 
    room_type : enum('Deluxe', 'Suite')
    """

In [3]:
@schema
class RoomAvailable(dj.Manual):
    definition = """
    -> Room
    date : date 
    ---
    price : decimal(6, 2)
    """

In [4]:
@schema
class Guest(dj.Manual):
    definition = """
    guest_id : int unsigned
    --- 
    guest_name : varchar(60)
    index(guest_name)
    """

In [5]:
@schema
class Reservation(dj.Manual):
    definition = """
    -> RoomAvailable
    ---
    -> Guest
    credit_card : varchar(80)
    """

In [6]:
@schema
class CheckIn(dj.Manual):
    definition = """
    -> Reservation
    """

In [7]:
@schema
class CheckOut(dj.Manual):
    definition = """
    -> CheckIn
    """

In [ ]:
dj.Diagram(schema)

# Populate Room Availability

In [8]:
import faker
import random
import datetime
import tqdm
fake = faker.Faker()

In [ ]:
# populate rooms
Room.insert((i, "Deluxe" if i%2 else "Suite") for i in range(80))

In [9]:
Room()

room,room_type
0,Suite
1,Deluxe
2,Suite
3,Deluxe
4,Suite
5,Deluxe
6,Suite
7,Deluxe
8,Suite
9,Deluxe


In [ ]:
# Populate Room availability: 45 days starting on start_date
start_date = datetime.date(2021, 10, 1)
days = 45

for day in tqdm.tqdm(range(days)):
    price = random.randint(40, 350) 
    RoomAvailable.insert(
        dict(key, 
             date=start_date + datetime.timedelta(days=day), 
             price=price) for key in Room.fetch('KEY'))
    

In [10]:
RoomAvailable()

room,date,price
0,2021-10-01,245.00
0,2021-10-02,221.00
0,2021-10-03,302.00
0,2021-10-04,69.00
0,2021-10-05,46.00
0,2021-10-06,222.00
0,2021-10-07,348.00
0,2021-10-08,313.00
0,2021-10-09,231.00
0,2021-10-10,278.00


# Functions 

In [12]:
class HotelException(Exception): pass
class RoomUnavailable(HotelException): pass
class RoomAlreadyReserved(HotelException): pass
class AlreadyChecked(HotelException): pass
class NoReservation(HotelException): pass
class NotCheckedIn(HotelException): pass

In [13]:
def reserve_room(room, date, guest_name, credit_card):
    # lookup guest by name
    keys = (Guest & {'guest_name': guest_name}).fetch('KEY')
    
    if keys:
        # if multiple found, use the first, for example
        key = keys[0]  
    else:
        # if not registered before, create a new guest id
        key = dict(guest_id=random.randint(0, 2**32-1))
        Guest.insert1(dict(key, guest_name=guest_name))
    
    try:
        Reservation.insert1(
            dict(key, room=room, date=date, credit_card=credit_card))
    except dj.errors.DuplicateError:
        raise RoomAlreadyReserved(room, date.isoformat()) from None
    except dj.errors.IntegrityError:
        raise RoomUnavailable(room, date.isoformat()) from None

In [14]:
def check_in(room, date):
    try:
        CheckIn.insert1(dict(room=room, date=date))
    except dj.errors.DuplicateError:
        raise AlreadyChecked(room, date.isoformat()) from None
    except dj.errors.IntegrityError:
        raise NoReservation(room, date.isoformat()) from None

In [15]:
def check_out(room, date):
    try:
        CheckOut.insert1(dict(room=room, date=date))
    except dj.errors.DuplicateError:
        raise AlreadyChecked(room, date.isoformat()) from None
    except dj.errors.IntegrityError:
        raise NotCheckedIn(room, date.isoformat()) from None

# Operations

In [16]:
# make a bunch of random reservations

number_of_guests = 100
max_nights = 20

for i in tqdm.tqdm(range(number_of_guests)):
    guest = fake.name()
    credit_card=' '.join((fake.credit_card_number(), 
                          fake.credit_card_expire(), 
                          fake.credit_card_security_code()))
    
    for j in range(random.randint(1, max_nights)):
        date =  fake.date_between_dates(start_date, start_date+datetime.timedelta(days=45))
        room = random.randint(0, 80)
        try:
            reserve_room(room, date, guest, credit_card)
        except HotelException as e:
            print(repr(e))
            
# show successful reservations
Reservation()

  5%|▌         | 5/100 [00:04<01:24,  1.12it/s]

RoomUnavailable(80, '2021-10-20')


  6%|▌         | 6/100 [00:05<01:33,  1.01it/s]

RoomUnavailable(80, '2021-10-11')


  8%|▊         | 8/100 [00:07<01:20,  1.15it/s]

...

 99%|█████████▉| 99/100 [01:31<00:00,  1.38it/s]

RoomAlreadyReserved(11, '2021-10-23')
RoomAlreadyReserved(9, '2021-11-11')
RoomAlreadyReserved(76, '2021-10-20')


100%|██████████| 100/100 [01:33<00:00,  1.07it/s]

RoomAlreadyReserved(46, '2021-10-23')


room,date,guest_id,credit_card
0,2021-10-02,898893816,4044459589861 12/24 151
0,2021-10-15,562856505,3550907737506216 05/25 886
0,2021-11-02,3933978801,4745727688097823 01/24 327
1,2021-10-02,1808624821,2514602675240723 09/28 101
1,2021-10-10,4194318221,38913904915741 05/30 013
1,2021-10-14,3378561279,4560180112466432 04/29 668
1,2021-10-15,3867661164,6569678467049022 06/30 815
1,2021-10-28,2700401322,5103212968781423 04/28 098
1,2021-11-04,4194318221,38913904915741 05/30 013
1,2021-11-06,3505359514,4598442891388914 01/29 304


In [11]:
Reservation()

room,date,guest_id,credit_card
0,2021-10-02,898893816,4044459589861 12/24 151
0,2021-10-15,562856505,3550907737506216 05/25 886
0,2021-11-02,3933978801,4745727688097823 01/24 327
1,2021-10-02,1808624821,2514602675240723 09/28 101
1,2021-10-10,4194318221,38913904915741 05/30 013
1,2021-10-14,3378561279,4560180112466432 04/29 668
1,2021-10-15,3867661164,6569678467049022 06/30 815
1,2021-10-28,2700401322,5103212968781423 04/28 098
1,2021-11-04,4194318221,38913904915741 05/30 013
1,2021-11-06,3505359514,4598442891388914 01/29 304


In [12]:
Guest()

guest_id,guest_name
1478700971,Aaron Meza
25213642,Amy Harper
898893816,Amy Robles
562856505,Amy Smith
949122841,Andrea Brock
1755680086,Andrew Dominguez
539272437,Andrew Estrada
829003304,Angela Barrett MD
823505441,Angela Davenport
3103647067,Ann Bond


In [18]:
# Try check in

check_in(2, datetime.date(2021, 10, 2))

NoReservation: (2, '2021-10-02')

In [24]:
# checkin a bunch of people
checkins = random.sample(Reservation().fetch('KEY'), k=int(0.9*len(Reservation())))
for r in tqdm.tqdm(checkins):
    check_in(**r)

100%|██████████| 781/781 [00:39<00:00, 19.70it/s]


In [13]:
CheckIn()

room,date
0,2021-10-02
0,2021-10-15
0,2021-11-02
1,2021-10-02
1,2021-10-10
1,2021-10-14
1,2021-10-15
1,2021-10-28
1,2021-11-04
1,2021-11-06


In [14]:
CheckOut()

room,date
0,2021-10-02
0,2021-10-15
0,2021-11-02
1,2021-10-02
1,2021-10-10
1,2021-10-14
1,2021-10-15
1,2021-10-28
1,2021-11-04
1,2021-11-06


In [26]:
# Try duplicate checkin
check_in(**checkins[0])

AlreadyChecked: (51, '2021-10-20')

In [27]:
# Try checkout

check_out(2, datetime.date(2021, 10, 2))

NotCheckedIn: (2, '2021-10-02')

In [28]:
# checkout a bunch of people
checkouts = random.sample(CheckIn().fetch('KEY'), k=int(0.9*len(CheckIn())))
for r in tqdm.tqdm(checkouts):
    check_out(**r)

100%|██████████| 702/702 [00:35<00:00, 19.78it/s]


In [29]:
# try duplicate checkout

check_out(**checkouts[0])

AlreadyChecked: (75, '2021-11-02')

# Queries 

## Query 1: List all guests who have stayed in room 1

In [30]:
Guest & (Reservation & (CheckIn & 'room=1'))

guest_id,guest_name
1808624821,Christine Ford
4194318221,Paul Conrad
3378561279,Samantha Smith
3867661164,Ricky Cain
2700401322,Kurt Patterson
3505359514,Elizabeth George
898893816,Amy Robles
265443910,Laura Hardy PhD
3922295966,Sarah Macias
1377498507,Jeffery Herrera


In [31]:
Guest & (Reservation * CheckIn & 'room=1')

guest_id,guest_name
1808624821,Christine Ford
4194318221,Paul Conrad
3378561279,Samantha Smith
3867661164,Ricky Cain
2700401322,Kurt Patterson
3505359514,Elizabeth George
898893816,Amy Robles
265443910,Laura Hardy PhD
3922295966,Sarah Macias
1377498507,Jeffery Herrera


## Query 2: List all nights when a guest stayed at a hotel

In [17]:
# pick a guest
guest = random.choice(Guest().fetch('KEY'))

In [18]:
guest

{'guest_id': 1105274806}

In [19]:
(Reservation * CheckIn & guest).proj()

room,date
7,2021-10-01
13,2021-10-03
15,2021-10-09
23,2021-11-13
28,2021-10-07
29,2021-11-13
41,2021-10-29
47,2021-10-29
62,2021-10-16
